# Experimental results

With this file I print the results as in the exprimental section. Because sometimes it dies due to memory, I saved the results in three pickles filed that you can load by setting the next parameter to True

In [6]:
use_saved_results = True

In [3]:
from ultralytics import YOLO
import numpy as np
import pickle
import torch
import gc

In [4]:
# The 300 epoch finetuned yolo
path_to_finetuned_checkpoint = "/home/edoardo/aiproject/runs/detect/300/weights/best.pt"
model_tuned = YOLO(path_to_finetuned_checkpoint)

np.random.seed(100)

torch.cuda.empty_cache()
gc.collect()

9

### Save inference data (divided by noise type, due to kernel memory issues)

Clean results

In [5]:
if not use_saved_results:
    results_clean = {}
    # Clean dataset
    inference_clean = model_tuned.val(data="wildlife.yaml")
    info = {
        "precision": inference_clean.box.p,
        "recall": inference_clean.box.r,
        "mAP": inference_clean.box.all_ap,
        "confusion_matrix": inference_clean.confusion_matrix,
    }

    results_clean["clean"] = info.copy()
    with open("results_detection_clean_yolo.pckl", "wb") as file:
        pickle.dump(results_clean, file)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


100%|██████████| 755k/755k [00:00<00:00, 5.49MB/s]
val: Scanning /home/edoardo/aiproject/datasets/wildlife/valid/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00,  9.80it/s]


                   all        225        379      0.942       0.91      0.953      0.827
               buffalo         62         89      0.951      0.899      0.948      0.824
              elephant         53         91      0.909      0.876      0.931      0.802
                 rhino         55         85      0.965      0.963      0.975      0.873
                 zebra         59        114      0.945      0.903      0.959      0.808
Speed: 0.5ms preprocess, 2.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val64


Gaussian results

In [3]:
if not use_saved_results:
    results_gaussian = {}

    # Noisy dataset
    noises = {"gaussian": [0.1, 0.2], "bernoulli": [0.1, 0.3]}
    for noise_type in ["gaussian"]:
        results_gaussian[noise_type] = dict()
        for noise in noises[noise_type]:
            results_gaussian[noise_type][noise] = dict()
            for model in ["v0", "v1"]:
                inference_data = model_tuned.val(
                    data=f"wildlife_{noise_type}_{noise}_{model}.yaml"
                )
                # Info to save
                info = {
                    "precision": inference_data.box.p,
                    "recall": inference_data.box.r,
                    "mAP": inference_data.box.all_ap,
                    "confusion_matrix": inference_data.confusion_matrix,
                }
                results_gaussian[noise_type][noise][model] = info.copy()

    with open("results_detection_gaussian_yolo.pckl", "wb") as file:
        pickle.dump(results_gaussian, file)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/edoardo/aiproject/datasets/wildlife/valid/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/15 [00:00<?, ?it/s]/home/edoardo/anaconda3/envs/aiproject/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 13.93it/s]


                   all        225        379      0.942       0.91      0.953      0.827
               buffalo         62         89      0.951      0.899      0.948      0.824
              elephant         53         91      0.909      0.876      0.931      0.802
                 rhino         55         85      0.965      0.963      0.975      0.873
                 zebra         59        114      0.945      0.903      0.959      0.808
Speed: 0.5ms preprocess, 2.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val50
Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)


val: Scanning /home/edoardo/aiproject/datasets/wildlife_gaussian_0.1_v0/valid/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 13.65it/s]


                   all        225        379      0.911      0.746      0.842      0.676
               buffalo         62         89      0.949      0.742      0.876      0.735
              elephant         53         91      0.818      0.741      0.775       0.62
                 rhino         55         85      0.891      0.882      0.931       0.77
                 zebra         59        114      0.986       0.62      0.785      0.581
Speed: 0.5ms preprocess, 1.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val51
Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)


...

                   all        225        379        0.9       0.75      0.835      0.672
               buffalo         62         89      0.867      0.854      0.893      0.752
              elephant         53         91      0.874      0.637      0.752      0.601
                 rhino         55         85      0.903      0.918      0.939      0.781
                 zebra         59        114      0.958      0.593      0.756      0.554
Speed: 0.5ms preprocess, 1.7ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val52
Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)


val: Scanning /home/edoardo/aiproject/datasets/wildlife_gaussian_0.2_v0/valid/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 13.26it/s]


                   all        225        379       0.87      0.693      0.786      0.606
               buffalo         62         89      0.937      0.673      0.798      0.667
              elephant         53         91        0.8      0.648       0.71      0.545
                 rhino         55         85      0.783      0.847      0.868      0.664
                 zebra         59        114      0.958      0.604      0.768      0.548
Speed: 0.5ms preprocess, 1.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val53
Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)


val: Scanning /home/edoardo/aiproject/datasets/wildlife_gaussian_0.2_v1/valid/labels.cache... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 12.17it/s]


                   all        225        379      0.804      0.617      0.705      0.513
               buffalo         62         89      0.775      0.695       0.77      0.601
              elephant         53         91      0.763      0.532      0.642      0.484
                 rhino         55         85      0.784      0.853       0.86      0.638
                 zebra         59        114      0.894      0.386      0.548      0.329
Speed: 0.6ms preprocess, 1.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val54


In [3]:
if not use_saved_results:
    results_bernoulli = {}
    # Noisy dataset
    noises = {"gaussian": [0.1, 0.2], "bernoulli": [0.1, 0.3]}
    for noise_type in ["bernoulli"]:
        results_bernoulli[noise_type] = dict()
        for noise in noises[noise_type]:
            results_bernoulli[noise_type][noise] = dict()
            for model in ["v0", "v1", "v2"]:
                inference_data = model_tuned.val(
                    data=f"wildlife_{noise_type}_{noise}_{model}.yaml"
                )
                # Info to save
                info = {
                    "precision": inference_data.box.p,
                    "recall": inference_data.box.r,
                    "mAP": inference_data.box.all_ap,
                    "confusion_matrix": inference_data.confusion_matrix,
                }
                results_bernoulli[noise_type][noise][model] = info.copy()

    with open("results_detection_bernoulli_yolo.pckl", "wb") as file:
        pickle.dump(results_bernoulli, file)

Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)
Model summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/edoardo/aiproject/datasets/wildlife_bernoulli_0.1_v0/valid/labels... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<00:00, 1680.95it/s]

val: New cache created: /home/edoardo/aiproject/datasets/wildlife_bernoulli_0.1_v0/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   7%|▋         | 1/15 [00:00<00:01,  8.43it/s]/home/edoardo/anaconda3/envs/aiproject/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 13.75it/s]


                   all        225        379      0.923      0.779      0.875      0.732
               buffalo         62         89      0.826      0.876      0.907      0.756
              elephant         53         91      0.951      0.648      0.815      0.674
                 rhino         55         85      0.962      0.882      0.949      0.822
                 zebra         59        114      0.953      0.711      0.828      0.676
Speed: 0.5ms preprocess, 2.8ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to runs/detect/val58
Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)


...

                   all        225        379      0.808      0.664       0.76      0.587
               buffalo         62         89      0.676      0.787       0.81      0.659
              elephant         53         91      0.804      0.495      0.671      0.502
                 rhino         55         85       0.86      0.867      0.893      0.703
                 zebra         59        114       0.89      0.509      0.665      0.482
Speed: 0.6ms preprocess, 1.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/detect/val62
Ultralytics YOLOv8.2.28 🚀 Python-3.10.0 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 7951MiB)


val: Scanning /home/edoardo/aiproject/datasets/wildlife_bernoulli_0.3_v2/valid/labels... 225 images, 0 backgrounds, 0 corrupt: 100%|██████████| 225/225 [00:00<00:00, 1898.90it/s]

val: New cache created: /home/edoardo/aiproject/datasets/wildlife_bernoulli_0.3_v2/valid/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:01<00:00, 12.88it/s]


                   all        225        379      0.727      0.516      0.615      0.443
               buffalo         62         89      0.518      0.685      0.656      0.507
              elephant         53         91      0.741      0.363       0.53      0.372
                 rhino         55         85      0.729      0.647       0.74      0.551
                 zebra         59        114       0.92      0.368      0.534      0.343
Speed: 0.5ms preprocess, 1.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/val63


### Load clean, Gaussian and Bernoulli results

In [7]:
with open("results_detection_clean_yolo.pckl", "rb") as file:
    results_clean = pickle.load(file)
with open("results_detection_gaussian_yolo.pckl", "rb") as file:
    results_gaussian = pickle.load(file)
with open("results_detection_bernoulli_yolo.pckl", "rb") as file:
    results_bernoulli = pickle.load(file)

In [8]:
np.set_printoptions(precision=3)
names = ["Buffalo", "Elephant", "Rhino", "Zebra"]
indexes = range(4)
# Clean dataset


# Simple utility just to print the matrixes. These functions are a bit complex and not really interesting
# They are just used to help me printing the latex tables
def get_res_matrix(result, noise_type, noises, model_kinds):
    matrixes = []
    for noise in noises:
        # V0
        matrix_kind = []
        for kind in model_kinds:
            res = result[noise_type][noise][kind]
            matr = np.zeros(
                (len(indexes) + 1, 3 * 1)
            )  # matrix that we update with the values to print
            matr[:, 0] = [
                res["precision"].mean(),
                *res["precision"],
            ]
            matr[:, 1] = [
                res["recall"].mean(),
                *res["recall"],
            ]
            # mAP50-95
            mAP50 = np.array([res["mAP"][j].mean() for j in indexes])
            matr[:, 2] = [
                mAP50.mean(),
                *mAP50,
            ]
            matrix_kind.append(matr)

        matrixes.append(np.hstack(matrix_kind))
    return np.vstack(matrixes)


# Baseline: the cleaned dataset

matr_cleaned = np.zeros((len(indexes) + 1, 3 * 1))

matr_cleaned[:, 0] = [
    results_clean["clean"]["precision"].mean(),
    *results_clean["clean"]["precision"],
]
matr_cleaned[:, 1] = [
    results_clean["clean"]["recall"].mean(),
    *results_clean["clean"]["recall"],
]
# mAP50-95
mAP50_c = np.array([results_clean["clean"]["mAP"][j].mean() for j in indexes])
matr_cleaned[:, 2] = [mAP50_c.mean(), *mAP50_c]
print(matr_cleaned)

# The Gaussian result matrix

res_matrix_gaussian = get_res_matrix(
    results_gaussian, "gaussian", [0.1, 0.2], ["v0", "v1"]
)

# The Bernoulli result matrix
res_matrix_bernoulli = get_res_matrix(
    results_bernoulli, "bernoulli", [0.1, 0.3], ["v2"]
)

[[0.942 0.91  0.827]
 [0.951 0.899 0.824]
 [0.909 0.876 0.802]
 [0.965 0.963 0.873]
 [0.945 0.903 0.808]]


Facility for printing the latex table of the report

In [7]:
# Latex table formatting
def latex_table_formatting(res_matrix):
    rows = [
        "All & 255 && ",
        "Buffalo & 89 && ",
        "Elephant & 91 && ",
        "Rhino & 85 && ",
        "Zebra & 114 &&",
    ]
    if len(res_matrix) > 5:
        rows += [
            "All & 255 && ",
            "Buffalo & 89 && ",
            "Elephant & 91 && ",
            "Rhino & 85 && ",
            "Zebra & 114 &&",
        ]
    for i in range(len(rows)):
        print(
            rows[i] + "& ".join([str(round(r, 3)) for r in res_matrix[i]]),
            end="\\\\ \n",
        )


latex_table_formatting(matr_cleaned)
print("")
latex_table_formatting(res_matrix_gaussian)
print("")
latex_table_formatting(res_matrix_bernoulli)

All & 255 && 0.942& 0.91& 0.827\\ 
Buffalo & 89 && 0.951& 0.899& 0.824\\ 
Elephant & 91 && 0.909& 0.876& 0.802\\ 
Rhino & 85 && 0.965& 0.963& 0.873\\ 
Zebra & 114 &&0.945& 0.903& 0.808\\ 

All & 255 && 0.911& 0.746& 0.676& 0.9& 0.75& 0.672\\ 
Buffalo & 89 && 0.949& 0.742& 0.735& 0.867& 0.854& 0.752\\ 
Elephant & 91 && 0.818& 0.741& 0.62& 0.874& 0.637& 0.601\\ 
Rhino & 85 && 0.891& 0.882& 0.77& 0.903& 0.918& 0.781\\ 
Zebra & 114 &&0.986& 0.62& 0.581& 0.958& 0.593& 0.554\\ 
All & 255 && 0.87& 0.693& 0.606& 0.804& 0.617& 0.513\\ 
Buffalo & 89 && 0.937& 0.673& 0.667& 0.775& 0.695& 0.601\\ 
Elephant & 91 && 0.8& 0.648& 0.545& 0.763& 0.532& 0.484\\ 
Rhino & 85 && 0.783& 0.847& 0.664& 0.784& 0.853& 0.638\\ 
Zebra & 114 &&0.958& 0.604& 0.548& 0.894& 0.386& 0.329\\ 

All & 255 && 0.866& 0.595& 0.541\\ 
Buffalo & 89 && 0.835& 0.738& 0.638\\ 
Elephant & 91 && 0.82& 0.449& 0.491\\ 
Rhino & 85 && 0.867& 0.8& 0.644\\ 
Zebra & 114 &&0.943& 0.395& 0.391\\ 
All & 255 && 0.727& 0.516& 0.443\\ 
Buffalo &

Facility for printing the confusion matrixes

In [43]:
# Columns are the ground truth (in order: buffalo, elephant, rhino, zebra, background), the rows the prediction
def print_cf_matrix(res, names=names):
    for i, n in enumerate(names + ["Backgr."]):
        print(f"{n} & {' & '.join(list(res[i].astype(int).astype(str)))}\\\\")


# Clean
res = results_clean["clean"]["confusion_matrix"].matrix
print_cf_matrix(res)

print("Gaussian")
# Gaussian
noises = [0.1, 0.2]
models = ["v0", "v1"]
for n in noises:
    for m in models:
        print(f"\n{n} | {m}")
        res = results_gaussian["gaussian"][n][m]["confusion_matrix"].matrix
        print_cf_matrix(res)

print("\nBernoulli")
# Bernoulli
noises = [0.1, 0.3]
models = ["v0", "v1"]
for n in noises:
    for m in models:
        print(f"\n{n} | {m}")
        res = results_bernoulli["bernoulli"][n][m]["confusion_matrix"].matrix
        print_cf_matrix(res)

Buffalo & 81 & 0 & 0 & 0 & 4\\
Elephant & 3 & 82 & 1 & 1 & 8\\
Rhino & 0 & 0 & 80 & 1 & 6\\
Zebra & 0 & 0 & 1 & 104 & 14\\
Backgr. & 5 & 9 & 3 & 8 & 0\\
Gaussian

0.1 | v0
Buffalo & 68 & 0 & 0 & 0 & 3\\
Elephant & 6 & 69 & 1 & 6 & 6\\
Rhino & 0 & 1 & 77 & 2 & 5\\
Zebra & 0 & 0 & 0 & 71 & 4\\
Backgr. & 15 & 21 & 7 & 35 & 0\\

0.1 | v1
Buffalo & 76 & 1 & 2 & 1 & 5\\
Elephant & 2 & 56 & 1 & 4 & 1\\
Rhino & 0 & 4 & 76 & 2 & 2\\
Zebra & 0 & 0 & 0 & 66 & 1\\
Backgr. & 11 & 30 & 6 & 41 & 0\\

0.2 | v0
Buffalo & 58 & 0 & 1 & 0 & 3\\
Elephant & 6 & 57 & 1 & 3 & 3\\
Rhino & 2 & 2 & 69 & 4 & 6\\
Zebra & 0 & 0 & 0 & 68 & 1\\
Backgr. & 23 & 32 & 14 & 39 & 0\\

0.2 | v1
Buffalo & 60 & 3 & 1 & 1 & 8\\
Elephant & 3 & 42 & 1 & 3 & 3\\
Rhino & 0 & 5 & 71 & 2 & 8\\
Zebra & 0 & 0 & 0 & 39 & 1\\
Backgr. & 26 & 41 & 12 & 69 & 0\\

Bernoulli

0.1 | v0
Buffalo & 80 & 8 & 1 & 2 & 9\\
Elephant & 0 & 59 & 1 & 2 & 5\\
Rhino & 0 & 2 & 77 & 2 & 3\\
Zebra & 0 & 0 & 1 & 83 & 5\\
Backgr. & 9 & 22 & 5 & 25 & 0\\

0.1 |